<a href="https://colab.research.google.com/github/Andrey-Mel/Integration_prodact/blob/main/Integration_NN_in_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#В колабе я создам и обучу модель для cifar10 и сохраню в формате SavedModel.
#Это сделаю из-за своего слабого компа.
#А установлю docker и tensorflow servin у себя и буду туда уже делать запросы

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
import tempfile
import matplotlib.pyplot as plt
import os 
from tensorflow.keras.datasets import cifar10,


#будем отслеживать все ошибки
import subprocess

#версия tf
print(tf.__version__)

2.4.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Создаю датасет

In [ ]:
#создание обучающей и проверочной выборки
(xTrain, yTrain),(xTest, yTest) = cifar10.load_data()
#(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
#нормализуем значения пикселей до интервала от0 до 1
xTrain = xTrain/255
xTest = xTest/255

#приводим к виду 4D тензора
xTrain = xTrain.reshape(xTrain.shape[0], 32, 32, 3)
xTest = xTest.reshape(xTest.shape[0], 32, 32, 3)

#Название классов
class_names = ['самолет','автомобиль','птица','кот','олень','собака','лягушка','лошадь','корабль','грузовик']

print('\n размер обучающей выбборки: {}'.format(xTrain.shape))
print('размер тестовой выборки: {}'.format(xTest.shape))
print('количество классов {}'.format(len(class_names)))

170500096/170498071 [==============================] - 2s 0us/step

 размер обучающей выбборки: (50000, 32, 32, 3)
размер тестовой выборки: (10000, 32, 32, 3)
количество классов 10


**Модель**

In [ ]:
model = Sequential()
model.add(Conv2D(filters=8, kernel_size=3, activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(filters=8, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

#Обучение
model.fit(xTrain, yTrain, epochs=30)

#Проверка точности модели
test_locc, test_acc = model.evaluate(xTest, yTest)
print()
print('\n Точность на проверке: {}'.format(test_acc))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 8)         224       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                15690     
Total params: 16,498
Trainable params: 16,498
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1563/1563 [==============================] - 4s 2ms/step - loss: 1.8114 - accuracy: 0.3451
Epoch 2/30
156

**Сохраняю моделль.**

Чтобы загрузить модель в Tf Serving, модель необходимо сохранить в формате SavedModel, это создаст файл Protobuf, в папке с четко прописаным путем.

In [ ]:
MODEL_DIREC = '/content/my_model' #Создание временной директории для хранения модели
version = 1 #номер версии
export_path = os.path.join(MODEL_DIREC, str(version))
print("Путь экспорта = {}\n".format(export_path))

Путь экспорта = /content/my_model/1



In [ ]:
if os.path.isdir(export_path):
  print('\nМодель сохранена\n')
  !rm -r {export_path}

In [ ]:
#Сохраняю модель в формате SavedModel
tf.keras.models.save_model(
    model,
    export_path,
    overwrite = True,
    include_optimizer=True,
    save_format=None,
    signatures = None,
    options=None
)

print('\nСохраненная модель: ')
!ls -l {export_path}

INFO:tensorflow:Assets written to: /content/my_model/1/assets

Сохраненная модель: 
total 116
drwxr-xr-x 2 root root   4096 Feb 13 15:20 assets
-rw-r--r-- 1 root root 110083 Feb 13 15:20 saved_model.pb
drwxr-xr-x 2 root root   4096 Feb 13 15:20 variables


In [ ]:
#Протестирую нашу сохранненную модель
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['conv2d_2_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32, 32, 3)
        name: serving_default_conv2d_2_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0213 15:26:20.148871 140667532420992 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core

Дальше запускается локальный браузер и видим предсказание цифр